# 11/11/2018

### Variant calling practice SEQTEC2018

NATHALIA GRAF-GRACHET

http://iobio.io/cshl_practical

# SETUP

```
[nathalia@aws] $  cd workspace
[nathalia@aws] $  mkdir cshl-practical
[nathalia@aws] $  cd cshl-practical/
[nathalia@aws] $  mkdir reference
[nathalia@aws] $  mkdir analysis
[nathalia@aws] $  REF="/home/ubuntu/workspace/data/fasta/GRCh37/Homo_sapiens.GRCh37.75.dna_sm.primary_assembly"
[nathalia@aws] $  DATA="/home/ubuntu/workspace/data/utah"
[nathalia@aws] $  cd reference/
[nathalia@aws] $  cp ~/workspace/data/utah/chr20_fragment.fa .

[nathalia@aws] $  echo $REF
/home/ubuntu/workspace/data/fasta/GRCh37/Homo_sapiens.GRCh37.75.dna_sm.primary_assembly

```

# Prep the reference sequence

create an index

https://github.com/lh3/bwa.git

```
[nathalia@aws] $  bwa

Program: bwa (alignment via Burrows-Wheeler transformation)
Version: 0.7.17-r1194-dirty
Contact: Heng Li <lh3@sanger.ac.uk>

Usage:   bwa <command> [options]

Command: index         index sequences in the FASTA format
         mem           BWA-MEM algorithm
         fastmap       identify super-maximal exact matches
         pemerge       merge overlapping paired ends (EXPERIMENTAL)
         aln           gapped/ungapped alignment
         samse         generate alignment (single ended)
         sampe         generate alignment (paired ended)
         bwasw         BWA-SW for long queries

         shm           manage indices in shared memory
         fa2pac        convert FASTA to PAC format
         pac2bwt       generate BWT from PAC
         pac2bwtgen    alternative algorithm for generating BWT
         bwtupdate     update .bwt to the new format
         bwt2sa        generate SA from BWT and Occ

Note: To use BWA, you need to first index the genome with `bwa index'.
      There are three alignment algorithms in BWA: `mem', `bwasw', and
      `aln/samse/sampe'. If you are not sure which to use, try `bwa mem'
      first. Please `man ./bwa.1' for the manual.


[nathalia@aws] $  bwa index

Usage:   bwa index [options] <in.fasta>

Options: -a STR    BWT construction algorithm: bwtsw, is or rb2 [auto]
         -p STR    prefix of the index [same as fasta name]
         -b INT    block size for the bwtsw algorithm (effective with -a bwtsw) [10000000]
         -6        index files named as <in.fasta>.64.* instead of <in.fasta>.* 

Warning: `-a bwtsw' does not work for short genomes, while `-a is' and
         `-a div' do not work not for long genomes.



```

```
[nathalia@aws] $  bwa index -p chr20_fragment chr20_fragment.fa
[bwa_index] Pack FASTA... 0.00 sec
[bwa_index] Construct BWT for the packed sequence...
[bwa_index] 0.02 seconds elapse.
[bwa_index] Update BWT... 0.00 sec
[bwa_index] Pack forward-only FASTA... 0.00 sec
[bwa_index] Construct SA from BWT and Occ... 0.01 sec
[main] Version: 0.7.17-r1194-dirty
[main] CMD: bwa index -p chr20_fragment chr20_fragment.fa
[main] Real time: 0.039 sec; CPU: 0.027 sec
[nathalia@aws] $  lhtr
total 300K
-rwxr-xr-- 1 ubuntu ubuntu 100K Nov 11 15:13 chr20_fragment.fa
-rw-rw-r-- 1 ubuntu ubuntu  98K Nov 11 15:17 chr20_fragment.bwt
-rw-rw-r-- 1 ubuntu ubuntu  25K Nov 11 15:17 chr20_fragment.pac
-rw-rw-r-- 1 ubuntu ubuntu   76 Nov 11 15:17 chr20_fragment.ann
-rw-rw-r-- 1 ubuntu ubuntu   11 Nov 11 15:17 chr20_fragment.amb
-rw-rw-r-- 1 ubuntu ubuntu  49K Nov 11 15:17 chr20_fragment.sa
```

```
[nathalia@aws] $  ls -l $DATA
total 68188
-rwxr-xr-- 1 ubuntu ubuntu 10694300 Nov  8 18:51 NA12878_1_sample.fastq
-rwxr-xr-- 1 ubuntu ubuntu 10693786 Nov  8 18:51 NA12878_2_sample.fastq
-rwxr-xr-- 1 ubuntu ubuntu 10772596 Nov  8 18:51 NA12891_1_sample.fastq
-rwxr-xr-- 1 ubuntu ubuntu 10772200 Nov  8 18:51 NA12891_2_sample.fastq
-rwxr-xr-- 1 ubuntu ubuntu 11323228 Nov  8 18:51 NA12892_1_sample.fastq
-rwxr-xr-- 1 ubuntu ubuntu 11322974 Nov  8 18:51 NA12892_2_sample.fastq
-rwxr-xr-- 1 ubuntu ubuntu   101739 Nov  8 18:51 chr20_fragment.fa
-rwxr-xr-- 1 ubuntu ubuntu  4024499 Nov  8 18:51 sample.bam
```

NA is the most sequenced person in history. It's a she, and she is dead. There's also her parents sequence here in this folder.

# Align reads

```
[nathalia@aws] $  cd ../analysis/


[nathalia@aws] $  bwa mem

Usage: bwa mem [options] <idxbase> <in1.fq> [in2.fq]

Algorithm options:

       -t INT        number of threads [1]
       -k INT        minimum seed length [19]
       -w INT        band width for banded alignment [100]
       -d INT        off-diagonal X-dropoff [100]
       -r FLOAT      look for internal seeds inside a seed longer than {-k} * FLOAT [1.5]
       -y INT        seed occurrence for the 3rd round seeding [20]
       -c INT        skip seeds with more than INT occurrences [500]
       -D FLOAT      drop chains shorter than FLOAT fraction of the longest overlapping chain [0.50]
       -W INT        discard a chain if seeded bases shorter than INT [0]
       -m INT        perform at most INT rounds of mate rescues for each read [50]
       -S            skip mate rescue
       -P            skip pairing; mate rescue performed unless -S also in use

Scoring options:

       -A INT        score for a sequence match, which scales options -TdBOELU unless overridden [1]
       -B INT        penalty for a mismatch [4]
       -O INT[,INT]  gap open penalties for deletions and insertions [6,6]
       -E INT[,INT]  gap extension penalty; a gap of size k cost '{-O} + {-E}*k' [1,1]
       -L INT[,INT]  penalty for 5'- and 3'-end clipping [5,5]
       -U INT        penalty for an unpaired read pair [17]

       -x STR        read type. Setting -x changes multiple parameters unless overridden [null]
                     pacbio: -k17 -W40 -r10 -A1 -B1 -O1 -E1 -L0  (PacBio reads to ref)
                     ont2d: -k14 -W20 -r10 -A1 -B1 -O1 -E1 -L0  (Oxford Nanopore 2D-reads to ref)
                     intractg: -B9 -O16 -L5  (intra-species contigs to ref)

Input/output options:

       -p            smart pairing (ignoring in2.fq)
       -R STR        read group header line such as '@RG\tID:foo\tSM:bar' [null]
       -H STR/FILE   insert STR to header if it starts with @; or insert lines in FILE [null]
       -o FILE       sam file to output results to [stdout]
       -j            treat ALT contigs as part of the primary assembly (i.e. ignore <idxbase>.alt file)
       -5            for split alignment, take the alignment with the smallest coordinate as primary
       -q            don't modify mapQ of supplementary alignments
       -K INT        process INT input bases in each batch regardless of nThreads (for reproducibility) []

       -v INT        verbosity level: 1=error, 2=warning, 3=message, 4+=debugging [3]
       -T INT        minimum score to output [30]
       -h INT[,INT]  if there are <INT hits with score >80% of the max score, output all in XA [5,200]
       -a            output all alignments for SE or unpaired PE
       -C            append FASTA/FASTQ comment to SAM output
       -V            output the reference FASTA header in the XR tag
       -Y            use soft clipping for supplementary alignments
       -M            mark shorter split hits as secondary

       -I FLOAT[,FLOAT[,INT[,INT]]]
                     specify the mean, standard deviation (10% of the mean if absent), max
                     (4 sigma from the mean if absent) and min of the insert size distribution.
                     FR orientation only. [inferred]

Note: Please read the man page for detailed description of the command line and options.

```

```
[nathalia@aws] $  lscpu
Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              8
On-line CPU(s) list: 0-7
Thread(s) per core:  2
Core(s) per socket:  4
Socket(s):           1
NUMA node(s):        1
Vendor ID:           GenuineIntel
CPU family:          6
Model:               85
Model name:          Intel(R) Xeon(R) Platinum 8175M CPU @ 2.50GHz
Stepping:            4
CPU MHz:             3128.053
BogoMIPS:            5000.00
Hypervisor vendor:   KVM
Virtualization type: full
L1d cache:           32K
L1i cache:           32K
L2 cache:            1024K
L3 cache:            33792K
NUMA node0 CPU(s):   0-7
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid aperfmperf pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt tsc_deadline_timer aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single pti fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm mpx avx512f avx512dq rdseed adx smap clflushopt clwb avx512cd avx512bw avx512vl xsaveopt xsavec xgetbv1 xsaves ida arat pku ospke

```

```
[nathalia@aws] $  bwa mem -t 8 -R "@RG\tID:NABOB\tPL:ILLUMINA\tSM:NA12878" ../reference/chr20_fragment /home/ubuntu/workspace/data/utah//NA12878_1_sample.fastq /home/ubuntu/workspace/data/utah//NA12878_2_sample.fastq | samblaster | samtools view -b -S - | samtools sort -o output.bam -
[M::bwa_idx_load_from_disk] read 0 ALT contigs
samblaster: Version 0.1.24
samblaster: Inputting from stdin
samblaster: Outputting to stdout
[M::process] read 93400 sequences (9433143 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (1, 26, 4, 8)
[M::mem_pestat] skip orientation FF as there are not enough pairs
[M::mem_pestat] analyzing insert size distribution for orientation FR...
[M::mem_pestat] (25, 50, 75) percentile: (93, 134, 2946)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (1, 8652)
[M::mem_pestat] mean and std.dev: (1139.08, 1689.00)
[M::mem_pestat] low and high boundaries for proper pairs: (1, 11505)
[M::mem_pestat] skip orientation RF as there are not enough pairs
[M::mem_pestat] skip orientation RR as there are not enough pairs
[M::mem_process_seqs] Processed 93400 reads in 18.184 CPU sec, 2.277 real sec
samblaster: Loaded 1 header sequence entries.
[main] Version: 0.7.17-r1194-dirty
[main] CMD: bwa mem -t 8 -R @RG\tID:NABOB\tPL:ILLUMINA\tSM:NA12878 ../reference/chr20_fragment /home/ubuntu/workspace/data/utah//NA12878_1_sample.fastq /home/ubuntu/workspace/data/utah//NA12878_2_sample.fastq
[main] Real time: 3.184 sec; CPU: 18.297 sec
samblaster: Marked 669 of 46700 (1.43%) read ids as duplicates using 3400k memory in 0.047S CPU seconds and 3S wall time.

[nathalia@aws] $  samtools index output.bam 


[nathalia@aws] $  lhtr
total 6.7M
-rw-rw-r-- 1 ubuntu ubuntu 6.7M Nov 11 15:28 output.bam
-rw-rw-r-- 1 ubuntu ubuntu  288 Nov 11 15:30 output.bam.bai

```

Yas it works... delete and repeat for 3 files with their respective names.

### tangent..

```
[nathalia@aws] $  sudo apt install tree
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 0 not upgraded.
Need to get 40.7 kB of archives.
After this operation, 105 kB of additional disk space will be used.
Get:1 http://us-west-2.ec2.archive.ubuntu.com/ubuntu bionic/universe amd64 tree amd64 1.7.0-5 [40.7 kB]
Fetched 40.7 kB in 0s (0 B/s)   
Selecting previously unselected package tree.
(Reading database ... 148761 files and directories currently installed.)
Preparing to unpack .../tree_1.7.0-5_amd64.deb ...
Unpacking tree (1.7.0-5) ...
Setting up tree (1.7.0-5) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
[nathalia@aws] $  tree
.
├── analysis
│   ├── output.bam
│   └── output.bam.bai
└── reference
    ├── chr20_fragment.amb
    ├── chr20_fragment.ann
    ├── chr20_fragment.bwt
    ├── chr20_fragment.fa
    ├── chr20_fragment.pac
    └── chr20_fragment.sa

2 directories, 8 files
```

### and back to it...

```
[nathalia@aws] $  screen -S screen1

#crtl+a and crtl+d = detaches
#crtl+d = terminates
[detached from 2639.pts-0.ip-172-31-39-185]


[nathalia@aws] $  screen -list
There are screens on:
	3240.screen1	(11/11/18 16:47:00)	(Detached)
1 Sockets in /run/screen/S-ubuntu.


[nathalia@aws] $  screen -r screen1 
[detached from 2674.screen1]
```

```
[nathalia@aws] $  rm -f output.bam*

[nathalia@aws] $  screen -S screen1
#need to set variables again in each screen...

[nathalia@aws] $  REF="/home/ubuntu/workspace/data/fasta/GRCh37/Homo_sapiens.GRCh37.75.dna_sm.primary_assembly"
[nathalia@aws] $  DATA="/home/ubuntu/workspace/data/utah/"

[nathalia@aws] $  bwa mem -t 8 -R "@RG\tID:NA12878\tPL:ILLUMINA\tSM:NA12878" $REF $DATA/NA12878_1_sample.fastq  $DATA/NA12878_2_sample.fastq |samblaster |samtools view -b -S - | samtools sort -o NA12878.bam -
samblaster: Version 0.1.24
samblaster: Inputting from stdin
samblaster: Outputting to stdout
[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 93400 sequences (9433143 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (11, 44457, 0, 7)
[M::mem_pestat] analyzing insert size distribution for orientation FF...
[M::mem_pestat] (25, 50, 75) percentile: (66, 147, 235)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (1, 573)
[M::mem_pestat] mean and std.dev: (135.73, 79.80)
[M::mem_pestat] low and high boundaries for proper pairs: (1, 742)
[M::mem_pestat] analyzing insert size distribution for orientation FR...
[M::mem_pestat] (25, 50, 75) percentile: (278, 323, 368)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (98, 548)
[M::mem_pestat] mean and std.dev: (322.32, 71.71)
[M::mem_pestat] low and high boundaries for proper pairs: (8, 638)
[M::mem_pestat] skip orientation RF as there are not enough pairs
[M::mem_pestat] skip orientation RR as there are not enough pairs
[M::mem_pestat] skip orientation FF
[M::mem_process_seqs] Processed 93400 reads in 14.201 CPU sec, 1.781 real sec
samblaster: Loaded 84 header sequence entries.
[main] Version: 0.7.17-r1194-dirty
[main] CMD: bwa mem -tsamblaster: Marked 148 of 46700 (0.32%) read ids as duplicates 8 -R using 4196k memory in  @RG\tID:NA12878\tPL:ILLUMINA\tSM:NA128780.059S /home/ubuntu/workspace/data/fasta/GRCh37/Homo_sapiens.GRCh37.75.dna_sm.primary_assembly CPU seconds and  /home/ubuntu/workspace/data/utah//NA12878_1_sample.fastq5S /home/ubuntu/workspace/data/utah//NA12878_2_sample.fastq wall time.

[main] Real time: 5.412 sec; CPU: 16.895 sec

#crtl+a+d

[detached from 3240.screen1]


[nathalia@aws] $  screen -r screen2

[nathalia@aws] $  REF="/home/ubuntu/workspace/data/fasta/GRCh37/Homo_sapiens.GRCh37.75.dna_sm.primary_assembly"
[nathalia@aws] $  DATA="/home/ubuntu/workspace/data/utah/"

[nathalia@aws] $  bwa mem -t 8 -R "@RG\tID:NA12891\tPL:ILLUMINA\tSM:NA12891" $REF $DATA/NA12891_1_sample.fastq $DATA/NA12891_2_sample.fastq | samblaster |samtools view -b -S - | samtools sort -o NA12891.bam -
samblaster: Version 0.1.24
samblaster: Inputting from stdin
samblaster: Outputting to stdout
[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 94086 sequences (9502240 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (0, 44764, 0, 0)
[M::mem_pestat] skip orientation FF as there are not enough pairs
[M::mem_pestat] analyzing insert size distribution for orientation FR...
[M::mem_pestat] (25, 50, 75) percentile: (264, 302, 343)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (106, 501)
[M::mem_pestat] mean and std.dev: (303.38, 61.30)
[M::mem_pestat] low and high boundaries for proper pairs: (27, 580)
[M::mem_pestat] skip orientation RF as there are not enough pairs
[M::mem_pestat] skip orientation RR as there are not enough pairs
[M::mem_process_seqs] Processed 94086 reads in 14.442 CPU sec, 1.812 real sec
samblaster: Loaded 84 header sequence entries.
[main] Version: 0.7.17-r1194-dirty
samblaster: Marked 412 of 47043 (0.88%) read ids as duplicates[main] CMD: bwa mem -t 8 using 4228k memory in  -R @RG\tID:NA12891\tPL:ILLUMINA\tSM:NA128910.055S /home/ubuntu/workspace/data/fasta/GRCh37/Homo_sapiens.GRCh37.75.dna_sm.primary_assembly CPU seconds and  /home/ubuntu/workspace/data/utah//NA12891_1_sample.fastq6S /home/ubuntu/workspace/data/utah//NA12891_2_sample.fastq wall time.

[main] Real time: 5.670 sec; CPU: 17.151 sec


[detached from 2736.screen2]


[nathalia@aws] $  screen -r screen3 


[nathalia@aws] $  REF="/home/ubuntu/workspace/data/fasta/GRCh37/Homo_sapiens.GRCh37.75.dna_sm.primary_assembly"
[nathalia@aws] $  DATA="/home/ubuntu/workspace/data/utah/"

[nathalia@aws] $  bwa mem -t 8 -R "@RG\tID:NA12892\tPL:ILLUMINA\tSM:NA12892" $REF $DATA/NA12892_1_sample.fastq  $DATA/NA12892_2_sample.fastq | samblaster | samtools view -b -S - | samtools sort -o NA12892.bam -
samblaster: Version 0.1.24
samblaster: Inputting from stdin
samblaster: Outputting to stdout
[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 98894 sequences (9988033 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (2, 47343, 0, 1)
[M::mem_pestat] skip orientation FF as there are not enough pairs
[M::mem_pestat] analyzing insert size distribution for orientation FR...
[M::mem_pestat] (25, 50, 75) percentile: (267, 306, 348)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (105, 510)
[M::mem_pestat] mean and std.dev: (308.26, 62.21)
[M::mem_pestat] low and high boundaries for proper pairs: (24, 591)
[M::mem_pestat] skip orientation RF as there are not enough pairs
[M::mem_pestat] skip orientation RR as there are not enough pairs
[M::mem_process_seqs] Processed 98894 reads in 13.476 CPU sec, 1.691 real sec
samblaster: Loaded 84 header sequence entries.
[main] Version: 0.7.17-r1194-dirty
samblaster: Marked 243 of 49447 (0.49%) read ids as duplicates[main] CMD: bwa mem using 4224k memory in  -t 80.071S -R CPU seconds and  @RG\tID:NA12892\tPL:ILLUMINA\tSM:NA128926S /home/ubuntu/workspace/data/fasta/GRCh37/Homo_sapiens.GRCh37.75.dna_sm.primary_assembly wall time.
 /home/ubuntu/workspace/data/utah//NA12892_1_sample.fastq /home/ubuntu/workspace/data/utah//NA12892_2_sample.fastq
[main] Real time: 5.585 sec; CPU: 16.179 sec

[detached from 2749.screen3]
```

```
[nathalia@aws] $  lhtr
total 21M
-rw-rw-r-- 1 ubuntu ubuntu 7.4M Nov 11 16:50 NA12878.bam
-rw-rw-r-- 1 ubuntu ubuntu 6.5M Nov 11 16:53 NA12891.bam
-rw-rw-r-- 1 ubuntu ubuntu 6.9M Nov 11 16:55 NA12892.bam
```

`"@RG\tID:NA12891\tPL:ILLUMINA\tSM:NA12891"`  
`RG` is read group; which is not necessary equal to your sample id; read group is where the sequences come from  
`PL` is platform of sequence  
`SM` is sample

In the line, I typed the following:

`bwa mem -t 8 -R "@RG\tID:NA12878\tPL:ILLUMINA\tSM:NA12878" $REF $DATA/NA12878_1_sample.fastq $DATA/NA12878_2_sample.fastq |
samblaster |
samtools view -b -S - |
samtools sort -o NA12878.bam -`

the `$REF` and `$DATA` get expanded; that's why in the code here it's written all out.


```
[nathalia@aws] $  for file in *.bam; do samtools index $file; done

[nathalia@aws] $  lhtr
total 24M
-rw-rw-r-- 1 ubuntu ubuntu  7.4M Nov 11 16:50 NA12878.bam
-rw-rw-r-- 1 ubuntu ubuntu  6.5M Nov 11 16:53 NA12891.bam
-rw-rw-r-- 1 ubuntu ubuntu  6.9M Nov 11 16:55 NA12892.bam
-rw-rw-r-- 1 ubuntu ubuntu 1007K Nov 11 16:56 NA12878.bam.bai
-rw-rw-r-- 1 ubuntu ubuntu  803K Nov 11 16:56 NA12891.bam.bai
-rw-rw-r-- 1 ubuntu ubuntu  1.1M Nov 11 16:56 NA12892.bam.bai
```

# Alignment stats

```
[nathalia@aws] $  for file in *.bam; do output=${file/bam/stats}; samtools flagstat $file > $output; done
[nathalia@aws] $  lhtr
total 24M
-rw-rw-r-- 1 ubuntu ubuntu  7.4M Nov 11 16:50 NA12878.bam
-rw-rw-r-- 1 ubuntu ubuntu  6.5M Nov 11 16:53 NA12891.bam
-rw-rw-r-- 1 ubuntu ubuntu  6.9M Nov 11 16:55 NA12892.bam
-rw-rw-r-- 1 ubuntu ubuntu 1007K Nov 11 16:56 NA12878.bam.bai
-rw-rw-r-- 1 ubuntu ubuntu  803K Nov 11 16:56 NA12891.bam.bai
-rw-rw-r-- 1 ubuntu ubuntu  1.1M Nov 11 16:56 NA12892.bam.bai
-rw-rw-r-- 1 ubuntu ubuntu   412 Nov 11 16:57 NA12878.stats
-rw-rw-r-- 1 ubuntu ubuntu   412 Nov 11 16:57 NA12891.stats
-rw-rw-r-- 1 ubuntu ubuntu   412 Nov 11 16:57 NA12892.stats

```

```
[nathalia@aws] $  for file in *.stats; do cat $file; echo '';done
93479 + 0 in total (QC-passed reads + QC-failed reads)
0 + 0 secondary
79 + 0 supplementary
296 + 0 duplicates
93150 + 0 mapped (99.65% : N/A)
93400 + 0 paired in sequencing
46700 + 0 read1
46700 + 0 read2
92358 + 0 properly paired (98.88% : N/A)
92742 + 0 with itself and mate mapped
329 + 0 singletons (0.35% : N/A)
4 + 0 with mate mapped to a different chr
1 + 0 with mate mapped to a different chr (mapQ>=5)

94151 + 0 in total (QC-passed reads + QC-failed reads)
0 + 0 secondary
65 + 0 supplementary
824 + 0 duplicates
93970 + 0 mapped (99.81% : N/A)
94086 + 0 paired in sequencing
47043 + 0 read1
47043 + 0 read2
92728 + 0 properly paired (98.56% : N/A)
93728 + 0 with itself and mate mapped
177 + 0 singletons (0.19% : N/A)
4 + 0 with mate mapped to a different chr
2 + 0 with mate mapped to a different chr (mapQ>=5)

98961 + 0 in total (QC-passed reads + QC-failed reads)
0 + 0 secondary
67 + 0 supplementary
486 + 0 duplicates
98795 + 0 mapped (99.83% : N/A)
98894 + 0 paired in sequencing
49447 + 0 read1
49447 + 0 read2
97956 + 0 properly paired (99.05% : N/A)
98564 + 0 with itself and mate mapped
164 + 0 singletons (0.17% : N/A)
2 + 0 with mate mapped to a different chr
1 + 0 with mate mapped to a different chr (mapQ>=5)


```

Checking the BAM files

http://bam.iobio.io

Choose BAM URL > Add this URL: https://s3.amazonaws.com/iobio/1000gSV/NA19238.mapped.ILLUMINA.bwa.YRI.exome.20130415.bam > Hit Go.

![iobio.bam visualization](bam_iobio_visualiz.png)

# Call variants

```
[nathalia@aws] $  freebayes -h
usage: freebayes [OPTION] ... [BAM FILE] ... 

Bayesian haplotype-based polymorphism discovery.

citation: Erik Garrison, Gabor Marth
          "Haplotype-based variant detection from short-read sequencing"
          arXiv:1207.3907 (http://arxiv.org/abs/1207.3907)

overview:

    To call variants from aligned short-read sequencing data, supply BAM files and
    a reference.  FreeBayes will provide VCF output on standard out describing SNPs,
    indels, and complex variants in samples in the input alignments.

    By default, FreeBayes will consider variants supported by at least 2
    observations in a single sample (-C) and also by at least 20% of the reads from
    a single sample (-F).  These settings are suitable to low to high depth
    sequencing in haploid and diploid samples, but users working with polyploid or
    pooled samples may wish to adjust them depending on the characteristics of
    their sequencing data.

    FreeBayes is capable of calling variant haplotypes shorter than a read length
    where multiple polymorphisms segregate on the same read.  The maximum distance
    between polymorphisms phased in this way is determined by the
    --max-complex-gap, which defaults to 3bp.  In practice, this can comfortably be
    set to half the read length.

    Ploidy may be set to any level (-p), but by default all samples are assumed to
    be diploid.  FreeBayes can model per-sample and per-region variation in
    copy-number (-A) using a copy-number variation map.

    FreeBayes can act as a frequency-based pooled caller and describe variants
    and haplotypes in terms of observation frequency rather than called genotypes.
    To do so, use --pooled-continuous and set input filters to a suitable level.
    Allele observation counts will be described by AO and RO fields in the VCF output.


examples:

    # call variants assuming a diploid sample
    freebayes -f ref.fa aln.bam >var.vcf

    # call variants assuming a diploid sample, providing gVCF output
    freebayes -f ref.fa --gvcf aln.bam >var.gvcf

    # require at least 5 supporting observations to consider a variant
    freebayes -f ref.fa -C 5 aln.bam >var.vcf

    # use a different ploidy
    freebayes -f ref.fa -p 4 aln.bam >var.vcf

    # assume a pooled sample with a known number of genome copies
    freebayes -f ref.fa -p 20 --pooled-discrete aln.bam >var.vcf

    # generate frequency-based calls for all variants passing input thresholds
    freebayes -f ref.fa -F 0.01 -C 1 --pooled-continuous aln.bam >var.vcf

    # use an input VCF (bgzipped + tabix indexed) to force calls at particular alleles
    freebayes -f ref.fa -@ in.vcf.gz aln.bam >var.vcf

    # generate long haplotype calls over known variants
    freebayes -f ref.fa --haplotype-basis-alleles in.vcf.gz \ 
                        --haplotype-length 50 aln.bam

    # naive variant calling: simply annotate observation counts of SNPs and indels
    freebayes -f ref.fa --haplotype-length 0 --min-alternate-count 1 \ 
        --min-alternate-fraction 0 --pooled-continuous --report-monomorphic >var.vcf


parameters:

   -h --help       Prints this help dialog.
   --version       Prints the release number and the git commit id.

input:

   -b --bam FILE   Add FILE to the set of BAM files to be analyzed.
   -L --bam-list FILE
                   A file containing a list of BAM files to be analyzed.
   -c --stdin      Read BAM input on stdin.
   -f --fasta-reference FILE
                   Use FILE as the reference sequence for analysis.
                   An index file (FILE.fai) will be created if none exists.
                   If neither --targets nor --region are specified, FreeBayes
                   will analyze every position in this reference.
   -t --targets FILE
                   Limit analysis to targets listed in the BED-format FILE.
   -r --region <chrom>:<start_position>-<end_position>
                   Limit analysis to the specified region, 0-base coordinates,
                   end_position not included (same as BED format).
                   Either '-' or '..' maybe used as a separator.
   -s --samples FILE
                   Limit analysis to samples listed (one per line) in the FILE.
                   By default FreeBayes will analyze all samples in its input
                   BAM files.
   --populations FILE
                   Each line of FILE should list a sample and a population which
                   it is part of.  The population-based bayesian inference model
                   will then be partitioned on the basis of the populations.
   -A --cnv-map FILE
                   Read a copy number map from the BED file FILE, which has
                   either a sample-level ploidy:
                      sample name, copy number
                   or a region-specific format:
                      reference sequence, start, end, sample name, copy number
                   ... for each region in each sample which does not have the
                   default copy number as set by --ploidy.

output:

   -v --vcf FILE   Output VCF-format results to FILE. (default: stdout)
   --gvcf
                   Write gVCF output, which indicates coverage in uncalled regions.
   --gvcf-chunk NUM
                   When writing gVCF output emit a record for every NUM bases.
   -@ --variant-input VCF
                   Use variants reported in VCF file as input to the algorithm.
                   Variants in this file will included in the output even if
                   there is not enough support in the data to pass input filters.
   -l --only-use-input-alleles
                   Only provide variant calls and genotype likelihoods for sites
                   and alleles which are provided in the VCF input, and provide
                   output in the VCF for all input alleles, not just those which
                   have support in the data.
   --haplotype-basis-alleles VCF
                   When specified, only variant alleles provided in this input
                   VCF will be used for the construction of complex or haplotype
                   alleles.
   --report-all-haplotype-alleles
                   At sites where genotypes are made over haplotype alleles,
                   provide information about all alleles in output, not only
                   those which are called.
   --report-monomorphic
                   Report even loci which appear to be monomorphic, and report all
                   considered alleles, even those which are not in called genotypes.
                   Loci which do not have any potential alternates have '.' for ALT.
   -P --pvar N     Report sites if the probability that there is a polymorphism
                   at the site is greater than N.  default: 0.0.  Note that post-
                   filtering is generally recommended over the use of this parameter.
   --strict-vcf
                   Generate strict VCF format (FORMAT/GQ will be an int)

population model:

   -T --theta N    The expected mutation rate or pairwise nucleotide diversity
                   among the population under analysis.  This serves as the
                   single parameter to the Ewens Sampling Formula prior model
                   default: 0.001
   -p --ploidy N   Sets the default ploidy for the analysis to N.  default: 2
   -J --pooled-discrete
                   Assume that samples result from pooled sequencing.
                   Model pooled samples using discrete genotypes across pools.
                   When using this flag, set --ploidy to the number of
                   alleles in each sample or use the --cnv-map to define
                   per-sample ploidy.
   -K --pooled-continuous
                   Output all alleles which pass input filters, regardles of
                   genotyping outcome or model.

reference allele:

   -Z --use-reference-allele
                   This flag includes the reference allele in the analysis as
                   if it is another sample from the same population.
   --reference-quality MQ,BQ
                   Assign mapping quality of MQ to the reference allele at each
                   site and base quality of BQ.  default: 100,60

allele scope:

   -n --use-best-n-alleles N
                   Evaluate only the best N SNP alleles, ranked by sum of
                   supporting quality scores.  (Set to 0 to use all; default: all)
   -E --max-complex-gap N
      --haplotype-length N
                   Allow haplotype calls with contiguous embedded matches of up
                   to this length. Set N=-1 to disable clumping. (default: 3)
   --min-repeat-size N
                   When assembling observations across repeats, require the total repeat
                   length at least this many bp.  (default: 5)
   --min-repeat-entropy N
                   To detect interrupted repeats, build across sequence until it has
                   entropy > N bits per bp. Set to 0 to turn off. (default: 1)
   --no-partial-observations
                   Exclude observations which do not fully span the dynamically-determined
                   detection window.  (default, use all observations, dividing partial
                   support across matching haplotypes when generating haplotypes.)

  These flags are meant for testing.
  They are not meant for filtering the output.
  They actually filter the input to the algorithm by throwing away alignments.
  This hurts performance by hiding information from the Bayesian model.
  Do not use them unless you can validate that they improve results!

   -I --throw-away-snp-obs     Remove SNP observations from input.
   -i --throw-away-indels-obs  Remove indel observations from input.
   -X --throw-away-mnp-obs     Remove MNP observations from input.
   -u --throw-away-complex-obs Remove complex allele observations from input.

  If you need to break apart haplotype calls to obtain one class of alleles,
  run the call with default parameters, then normalize and subset the VCF:
    freebayes ... | vcfallelicprimitives -kg >calls.vcf
  For example, this would retain only biallelic SNPs.
    <calls.vcf vcfsnps | vcfbiallelic >biallelic_snp_calls.vcf

indel realignment:

   -O --dont-left-align-indels
                   Turn off left-alignment of indels, which is enabled by default.

input filters:

   -4 --use-duplicate-reads
                   Include duplicate-marked alignments in the analysis.
                   default: exclude duplicates marked as such in alignments
   -m --min-mapping-quality Q
                   Exclude alignments from analysis if they have a mapping
                   quality less than Q.  default: 1
   -q --min-base-quality Q
                   Exclude alleles from analysis if their supporting base
                   quality is less than Q.  default: 0
   -R --min-supporting-allele-qsum Q
                   Consider any allele in which the sum of qualities of supporting
                   observations is at least Q.  default: 0
   -Y --min-supporting-mapping-qsum Q
                   Consider any allele in which and the sum of mapping qualities of
                   supporting reads is at least Q.  default: 0
   -Q --mismatch-base-quality-threshold Q
                   Count mismatches toward --read-mismatch-limit if the base
                   quality of the mismatch is >= Q.  default: 10
   -U --read-mismatch-limit N
                   Exclude reads with more than N mismatches where each mismatch
                   has base quality >= mismatch-base-quality-threshold.
                   default: ~unbounded
   -z --read-max-mismatch-fraction N
                   Exclude reads with more than N [0,1] fraction of mismatches where
                   each mismatch has base quality >= mismatch-base-quality-threshold
                   default: 1.0
   -$ --read-snp-limit N
                   Exclude reads with more than N base mismatches, ignoring gaps
                   with quality >= mismatch-base-quality-threshold.
                   default: ~unbounded
   -e --read-indel-limit N
                   Exclude reads with more than N separate gaps.
                   default: ~unbounded
   -0 --standard-filters  Use stringent input base and mapping quality filters
                   Equivalent to -m 30 -q 20 -R 0 -S 0
   -F --min-alternate-fraction N
                   Require at least this fraction of observations supporting
                   an alternate allele within a single individual in the
                   in order to evaluate the position.  default: 0.05
   -C --min-alternate-count N
                   Require at least this count of observations supporting
                   an alternate allele within a single individual in order
                   to evaluate the position.  default: 2
   -3 --min-alternate-qsum N
                   Require at least this sum of quality of observations supporting
                   an alternate allele within a single individual in order
                   to evaluate the position.  default: 0
   -G --min-alternate-total N
                   Require at least this count of observations supporting
                   an alternate allele within the total population in order
                   to use the allele in analysis.  default: 1
   --min-coverage N
                   Require at least this coverage to process a site. default: 0
   --max-coverage N
                   Do not process sites with greater than this coverage. default: no limit

population priors:

   -k --no-population-priors
                   Equivalent to --pooled-discrete --hwe-priors-off and removal of
                   Ewens Sampling Formula component of priors.

mappability priors:

   -w --hwe-priors-off
                   Disable estimation of the probability of the combination
                   arising under HWE given the allele frequency as estimated
                   by observation frequency.
   -V --binomial-obs-priors-off
                   Disable incorporation of prior expectations about observations.
                   Uses read placement probability, strand balance probability,
                   and read position (5'-3') probability.
   -a --allele-balance-priors-off
                   Disable use of aggregate probability of observation balance between alleles
                   as a component of the priors.

genotype likelihoods:

   --observation-bias FILE
                   Read length-dependent allele observation biases from FILE.
                   The format is [length] [alignment efficiency relative to reference]
                   where the efficiency is 1 if there is no relative observation bias.
   --base-quality-cap Q
                   Limit estimated observation quality by capping base quality at Q.
   --prob-contamination F
                   An estimate of contamination to use for all samples.  default: 10e-9
   --legacy-gls    Use legacy (polybayes equivalent) genotype likelihood calculations
   --contamination-estimates FILE
                   A file containing per-sample estimates of contamination, such as
                   those generated by VerifyBamID.  The format should be:
                       sample p(read=R|genotype=AR) p(read=A|genotype=AA)
                   Sample '*' can be used to set default contamination estimates.

algorithmic features:

   --report-genotype-likelihood-max
                   Report genotypes using the maximum-likelihood estimate provided
                   from genotype likelihoods.
   -B --genotyping-max-iterations N
                   Iterate no more than N times during genotyping step. default: 1000.
   --genotyping-max-banddepth N
                   Integrate no deeper than the Nth best genotype by likelihood when
                   genotyping. default: 6.
   -W --posterior-integration-limits N,M
                   Integrate all genotype combinations in our posterior space
                   which include no more than N samples with their Mth best
                   data likelihood. default: 1,3.
   -N --exclude-unobserved-genotypes
                   Skip sample genotypings for which the sample has no supporting reads.
   -S --genotype-variant-threshold N
                   Limit posterior integration to samples where the second-best
                   genotype likelihood is no more than log(N) from the highest
                   genotype likelihood for the sample.  default: ~unbounded
   -j --use-mapping-quality
                   Use mapping quality of alleles when calculating data likelihoods.
   -H --harmonic-indel-quality
                   Use a weighted sum of base qualities around an indel, scaled by the
                   distance from the indel.  By default use a minimum BQ in flanking sequence.
   -D --read-dependence-factor N
                   Incorporate non-independence of reads by scaling successive
                   observations by this factor during data likelihood
                   calculations.  default: 0.9
   -= --genotype-qualities
                   Calculate the marginal probability of genotypes and report as GQ in
                   each sample field in the VCF output.

debugging:

   -d --debug      Print debugging output.
   -dd             Print more verbose debugging output (requires "make DEBUG")


author:   Erik Garrison <erik.garrison@bc.edu>, Marth Lab, Boston College, 2010-2014
version:  v1.2.0-2-g29c4002
```

**Poor man multithreading**
```
[nathalia@aws] $  screen -r screen1

[nathalia@aws] $  freebayes -f $REF.fa -b NA12878.bam -b NA12891.bam -b NA12892.bam -r 17 -v chr17.vcf

# took approximately 2-4 mins

[detached from 3240.screen1]

[nathalia@aws] $  screen -r screen2

[nathalia@aws] $  freebayes -f $REF.fa -b NA12878.bam -b NA12891.bam -b NA12892.bam -r 20 -v chr20.vcf

[detached from 2736.screen2]

[nathalia@aws] $  screen -r screen3

[nathalia@aws] $  freebayes -f $REF.fa -b NA12878.bam -b NA12891.bam -b NA12892.bam -r 21 -v chr21.vcf

[detached from 2749.screen3]

[nathalia@aws] $  screen -r screen4

[nathalia@aws] $  REF="/home/ubuntu/workspace/data/fasta/GRCh37/Homo_sapiens.GRCh37.75.dna_sm.primary_assembly"
[nathalia@aws] $  DATA="/home/ubuntu/workspace/data/utah/"
[nathalia@aws] $  freebayes -f $REF.fa -b NA12878.bam -b NA12891.bam -b NA12892.bam -r 22 -v chr22.vcf

[detached from 3495.screen4]

```

Could have done with `nohup`

```
$ nohup freebayes -f $REF.fa -b NA12878.bam -b NA12891.bam -b NA12892.bam -r 17 -v chr17.vcf & 
$ nohup freebayes -f $REF.fa -b NA12878.bam -b NA12891.bam -b NA12892.bam -r 20 -v chr20.vcf &
$ nohup freebayes -f $REF.fa -b NA12878.bam -b NA12891.bam -b NA12892.bam -r 21 -v chr21.vcf &
$ nodup freebayes -f $REF.fa -b NA12878.bam -b NA12891.bam -b NA12892.bam -r 22 -v chr22.vcf &
```

```
[nathalia@aws] $  lhtr
total 25M
-rw-rw-r-- 1 ubuntu ubuntu  7.4M Nov 11 16:50 NA12878.bam
-rw-rw-r-- 1 ubuntu ubuntu  6.5M Nov 11 16:53 NA12891.bam
-rw-rw-r-- 1 ubuntu ubuntu  6.9M Nov 11 16:55 NA12892.bam
-rw-rw-r-- 1 ubuntu ubuntu 1007K Nov 11 16:56 NA12878.bam.bai
-rw-rw-r-- 1 ubuntu ubuntu  803K Nov 11 16:56 NA12891.bam.bai
-rw-rw-r-- 1 ubuntu ubuntu  1.1M Nov 11 16:56 NA12892.bam.bai
-rw-rw-r-- 1 ubuntu ubuntu   412 Nov 11 16:57 NA12878.stats
-rw-rw-r-- 1 ubuntu ubuntu   412 Nov 11 16:57 NA12891.stats
-rw-rw-r-- 1 ubuntu ubuntu   412 Nov 11 16:57 NA12892.stats
-rw-rw-r-- 1 ubuntu ubuntu  900K Nov 11 17:57 chr17.vcf
-rw-rw-r-- 1 ubuntu ubuntu   81K Nov 11 17:57 chr20.vcf
-rw-rw-r-- 1 ubuntu ubuntu  196K Nov 11 17:58 chr21.vcf
-rw-rw-r-- 1 ubuntu ubuntu  216K Nov 11 17:58 chr22.vcf
```

```
[nathalia@aws] $  bcftools concat -o all.vcf chr17.vcf chr20.vcf chr21.vcf chr22.vcf
[nathalia@aws] $  bgzip all.vcf
[nathalia@aws] $  bcftools index all.vcf.gz
[nathalia@aws] $  lhtr
total 26M
-rw-rw-r-- 1 ubuntu ubuntu  7.4M Nov 11 16:50 NA12878.bam
-rw-rw-r-- 1 ubuntu ubuntu  6.5M Nov 11 16:53 NA12891.bam
-rw-rw-r-- 1 ubuntu ubuntu  6.9M Nov 11 16:55 NA12892.bam
-rw-rw-r-- 1 ubuntu ubuntu 1007K Nov 11 16:56 NA12878.bam.bai
-rw-rw-r-- 1 ubuntu ubuntu  803K Nov 11 16:56 NA12891.bam.bai
-rw-rw-r-- 1 ubuntu ubuntu  1.1M Nov 11 16:56 NA12892.bam.bai
-rw-rw-r-- 1 ubuntu ubuntu   412 Nov 11 16:57 NA12878.stats
-rw-rw-r-- 1 ubuntu ubuntu   412 Nov 11 16:57 NA12891.stats
-rw-rw-r-- 1 ubuntu ubuntu   412 Nov 11 16:57 NA12892.stats
-rw-rw-r-- 1 ubuntu ubuntu  900K Nov 11 17:57 chr17.vcf
-rw-rw-r-- 1 ubuntu ubuntu   81K Nov 11 17:57 chr20.vcf
-rw-rw-r-- 1 ubuntu ubuntu  196K Nov 11 17:58 chr21.vcf
-rw-rw-r-- 1 ubuntu ubuntu  216K Nov 11 17:58 chr22.vcf
-rw-rw-r-- 1 ubuntu ubuntu  347K Nov 11 18:04 all.vcf.gz
-rw-rw-r-- 1 ubuntu ubuntu   352 Nov 11 18:04 all.vcf.gz.csi
```

# Check VCF files

https://vcf.iobio.io/?species=Human

Hit Choose VCF URL > Check that Human and GRCh37 are selected > Hit Load

![iobio.bam visualization](vcf_iobio_visualiz.png)

```
#local machine

[nathalia@cshl] $  cd Desktop/nathalia_seqtec18/11-11-2018/

[nathalia@cshl] $  ls
AaronsClass-3-30-17.pdf			bam_iobio_visualiz.png
CSHL-Seq-2017-Var-discovery.pdf		variant_calling_practice_andrew.ipynb
aaron_class_2017.pdf			vcf_iobio_visualiz.png

[nathalia@cshl] $  scp -i ~/cshl/cshl_2018_student.pem ubuntu@52.26.253.73:/home/ubuntu/workspace/cshl-practical/analysis/all.vcf.gz .
all.vcf.gz                                    100%  346KB 720.1KB/s   00:00    
```

Trying to load in the website...

Message: `must select 2 files, both a .vcf.gz and .vcf.gz.tbi file`

I don't have any `.tbi` file...
```
[nathalia@aws] $  lhtr
total 26M
-rw-rw-r-- 1 ubuntu ubuntu  7.4M Nov 11 16:50 NA12878.bam
-rw-rw-r-- 1 ubuntu ubuntu  6.5M Nov 11 16:53 NA12891.bam
-rw-rw-r-- 1 ubuntu ubuntu  6.9M Nov 11 16:55 NA12892.bam
-rw-rw-r-- 1 ubuntu ubuntu 1007K Nov 11 16:56 NA12878.bam.bai
-rw-rw-r-- 1 ubuntu ubuntu  803K Nov 11 16:56 NA12891.bam.bai
-rw-rw-r-- 1 ubuntu ubuntu  1.1M Nov 11 16:56 NA12892.bam.bai
-rw-rw-r-- 1 ubuntu ubuntu   412 Nov 11 16:57 NA12878.stats
-rw-rw-r-- 1 ubuntu ubuntu   412 Nov 11 16:57 NA12891.stats
-rw-rw-r-- 1 ubuntu ubuntu   412 Nov 11 16:57 NA12892.stats
-rw-rw-r-- 1 ubuntu ubuntu  900K Nov 11 17:57 chr17.vcf
-rw-rw-r-- 1 ubuntu ubuntu   81K Nov 11 17:57 chr20.vcf
-rw-rw-r-- 1 ubuntu ubuntu  196K Nov 11 17:58 chr21.vcf
-rw-rw-r-- 1 ubuntu ubuntu  216K Nov 11 17:58 chr22.vcf
-rw-rw-r-- 1 ubuntu ubuntu  347K Nov 11 18:04 all.vcf.gz
-rw-rw-r-- 1 ubuntu ubuntu   352 Nov 11 18:04 all.vcf.gz.csi


#testing with `.csi`

[nathalia@cshl] $  scp -i ~/cshl/cshl_2018_student.pem ubuntu@52.26.253.73:/home/ubuntu/workspace/cshl-practical/analysis/all.vcf.gz.csi .
all.vcf.gz.csi                                100%  352     4.3KB/s   00:00    


# https://www.biostars.org/p/59492/

[nathalia@aws] $  tabix -p vcf all.vcf.gz

[nathalia@aws] $  lhtr
total 26M
-rw-rw-r-- 1 ubuntu ubuntu  7.4M Nov 11 16:50 NA12878.bam
-rw-rw-r-- 1 ubuntu ubuntu  6.5M Nov 11 16:53 NA12891.bam
-rw-rw-r-- 1 ubuntu ubuntu  6.9M Nov 11 16:55 NA12892.bam
-rw-rw-r-- 1 ubuntu ubuntu 1007K Nov 11 16:56 NA12878.bam.bai
-rw-rw-r-- 1 ubuntu ubuntu  803K Nov 11 16:56 NA12891.bam.bai
-rw-rw-r-- 1 ubuntu ubuntu  1.1M Nov 11 16:56 NA12892.bam.bai
-rw-rw-r-- 1 ubuntu ubuntu   412 Nov 11 16:57 NA12878.stats
-rw-rw-r-- 1 ubuntu ubuntu   412 Nov 11 16:57 NA12891.stats
-rw-rw-r-- 1 ubuntu ubuntu   412 Nov 11 16:57 NA12892.stats
-rw-rw-r-- 1 ubuntu ubuntu  900K Nov 11 17:57 chr17.vcf
-rw-rw-r-- 1 ubuntu ubuntu   81K Nov 11 17:57 chr20.vcf
-rw-rw-r-- 1 ubuntu ubuntu  196K Nov 11 17:58 chr21.vcf
-rw-rw-r-- 1 ubuntu ubuntu  216K Nov 11 17:58 chr22.vcf
-rw-rw-r-- 1 ubuntu ubuntu  347K Nov 11 18:04 all.vcf.gz
-rw-rw-r-- 1 ubuntu ubuntu   352 Nov 11 18:04 all.vcf.gz.csi
-rw-rw-r-- 1 ubuntu ubuntu  116K Nov 11 18:40 NA12878.vcf
-rw-rw-r-- 1 ubuntu ubuntu   60K Nov 11 18:44 test.vcf
-rw-rw-r-- 1 ubuntu ubuntu   557 Nov 11 19:13 all.vcf.gz.tbi

#now I have the .csi

[nathalia@cshl] $  scp -i ~/cshl/cshl_2018_student.pem ubuntu@52.26.253.73:/home/ubuntu/workspace/cshl-practical/analysis/all.vcf.gz.tbi .
all.vcf.gz.tbi                                100%  557     6.8KB/s   00:00    

```

Went to the website: https://vcf.iobio.io/?species=Human&build=GRCh37 > Make sure Human and GRCh37 are selected > Click on Select All Samples > Hit Load

![Visualization with the files we used](practice_vcf_visualization.png)

The number of variants I had (252) didn't match with the girls around me. So I got the link to my public AWS and put it there.

http://ec2-52-26-253-73.us-west-2.compute.amazonaws.com/workspace/cshl-practical/analysis/

Right-click on `all.vcf.gz`

Paste this URL onto to the iobio, and now I have all 766 variants.

Seems like we still need the `.tbi` file, because the girls tried to upload their URL without the `.tbi` and it didn't load.


![now works](practice2_vcf_visialization.png)

# Call variants

```
[nathalia@aws] $  bcftools concat -o all.vcf chr17.vcf chr20.vcf chr21.vcf chr22.vcf
# the above step has t be done with chr from the same person

[nathalia@aws] $  bgzip all.vcf
# never use gzip bc then cannot index

[nathalia@aws] $  bcftools index all.vcf.gz

[nathalia@aws] $  lhtr
total 26M
-rw-rw-r-- 1 ubuntu ubuntu  7.4M Nov 11 16:50 NA12878.bam
-rw-rw-r-- 1 ubuntu ubuntu  6.5M Nov 11 16:53 NA12891.bam
-rw-rw-r-- 1 ubuntu ubuntu  6.9M Nov 11 16:55 NA12892.bam
-rw-rw-r-- 1 ubuntu ubuntu 1007K Nov 11 16:56 NA12878.bam.bai
-rw-rw-r-- 1 ubuntu ubuntu  803K Nov 11 16:56 NA12891.bam.bai
-rw-rw-r-- 1 ubuntu ubuntu  1.1M Nov 11 16:56 NA12892.bam.bai
-rw-rw-r-- 1 ubuntu ubuntu   412 Nov 11 16:57 NA12878.stats
-rw-rw-r-- 1 ubuntu ubuntu   412 Nov 11 16:57 NA12891.stats
-rw-rw-r-- 1 ubuntu ubuntu   412 Nov 11 16:57 NA12892.stats
-rw-rw-r-- 1 ubuntu ubuntu  900K Nov 11 17:57 chr17.vcf
-rw-rw-r-- 1 ubuntu ubuntu   81K Nov 11 17:57 chr20.vcf
-rw-rw-r-- 1 ubuntu ubuntu  196K Nov 11 17:58 chr21.vcf
-rw-rw-r-- 1 ubuntu ubuntu  216K Nov 11 17:58 chr22.vcf
-rw-rw-r-- 1 ubuntu ubuntu  347K Nov 11 18:04 all.vcf.gz
-rw-rw-r-- 1 ubuntu ubuntu   352 Nov 11 18:04 all.vcf.gz.csi
```

# Manipulate VCF files

```
[nathalia@aws] $  bcftools view all.vcf.gz 17 | less -S

[nathalia@aws] $  bcftools view -H all.vcf.gz 17:17585028-17585100 | less -S

[nathalia@aws] $  bcftools view -i 'TYPE="snp"' all.vcf.gz | less -S



[nathalia@aws] $  bcftools view all.vcf.gz 22:39636944 | grep -v ^## | less -S

#CHROM  POS     ID      REF     ALT     QUAL    FILTER  INFO    FORMAT  NA12892 NA12878 NA12891
22      39636944        .       ACACTC  AC,ACTCTC,ACACAC        1703.26 .       AB=0.632075,0.141509,



[nathalia@aws] $  bcftools view all.vcf.gz 22:39636944 | bcftools norm -m - | grep -v ^## | less -S

#CHROM  POS     ID      REF     ALT     QUAL    FILTER  INFO    FORMAT  NA12892 NA12878 NA12891
22      39636944        .       ACACTC  AC      1703.26 .       AB=0.632075;ABP=19.071;AC=3;AF=0.5;AN
22      39636944        .       ACACTC  ACTCTC  1703.26 .       AB=0.141509;ABP=121.335;AC=2;AF=0.333
22      39636944        .       ACACTC  ACACAC  1703.26 .       AB=0.207547;ABP=81.7569;AC=1;AF=0.166



[nathalia@aws] $  bcftools view all.vcf.gz 22:39636944 | bcftools norm -f $REF.fa -m - | grep -v ^## | less -S

#CHROM  POS     ID      REF     ALT     QUAL    FILTER  INFO    FORMAT  NA12892 NA12878 NA12891
22      39636944        .       ACACT   A       1703.26 .       AB=0.632075;ABP=19.071;AC=3;AF=0.5;AN
22      39636946        .       A       T       1703.26 .       AB=0.141509;ABP=121.335;AC=2;AF=0.333
22      39636948        .       T       A       1703.26 .       AB=0.207547;ABP=81.7569;AC=1;AF=0.166


```

Contents of vcf file explained: http://www.internationalgenome.org/wiki/Analysis/vcf4.0/

```
[nathalia@aws] $  zcat all.vcf.gz | grep -v ^## | head | column -t | less -S
#grep canot take compressed file
```

# Subsetting on samples

```
[nathalia@aws] $  bcftools view -s NA12878 all.vcf.gz | less -S

[nathalia@aws] $  bcftools view -s NA12878 all.vcf.gz | wc -l 
2632
[nathalia@aws] $  bcftools view all.vcf.gz | wc -l
2632

# These numbers are supposed to be the same because the files have the same number of lines; it doesn't represent the number of polymorphisms... this command only deleted the columns that match to the sample we selected

[nathalia@aws] $  bcftools view -s NA12878 all.vcf.gz | grep "0/0" | less -S

# not polymorphic

[nathalia@aws] $  bcftools view -s NA12878 all.vcf.gz | bcftools view -e 'GT=="RR"' -o NA12878.vcf -

# all sites that show polymorphism; 'GT=="RR"' Genotype == reference reference

[nathalia@aws] $  lhtr
total 26M
-rw-rw-r-- 1 ubuntu ubuntu  7.4M Nov 11 16:50 NA12878.bam
-rw-rw-r-- 1 ubuntu ubuntu  6.5M Nov 11 16:53 NA12891.bam
-rw-rw-r-- 1 ubuntu ubuntu  6.9M Nov 11 16:55 NA12892.bam
-rw-rw-r-- 1 ubuntu ubuntu 1007K Nov 11 16:56 NA12878.bam.bai
-rw-rw-r-- 1 ubuntu ubuntu  803K Nov 11 16:56 NA12891.bam.bai
-rw-rw-r-- 1 ubuntu ubuntu  1.1M Nov 11 16:56 NA12892.bam.bai
-rw-rw-r-- 1 ubuntu ubuntu   412 Nov 11 16:57 NA12878.stats
-rw-rw-r-- 1 ubuntu ubuntu   412 Nov 11 16:57 NA12891.stats
-rw-rw-r-- 1 ubuntu ubuntu   412 Nov 11 16:57 NA12892.stats
-rw-rw-r-- 1 ubuntu ubuntu  900K Nov 11 17:57 chr17.vcf
-rw-rw-r-- 1 ubuntu ubuntu   81K Nov 11 17:57 chr20.vcf
-rw-rw-r-- 1 ubuntu ubuntu  196K Nov 11 17:58 chr21.vcf
-rw-rw-r-- 1 ubuntu ubuntu  216K Nov 11 17:58 chr22.vcf
-rw-rw-r-- 1 ubuntu ubuntu  347K Nov 11 18:04 all.vcf.gz
-rw-rw-r-- 1 ubuntu ubuntu   352 Nov 11 18:04 all.vcf.gz.csi
-rw-rw-r-- 1 ubuntu ubuntu  116K Nov 11 18:40 NA12878.vcf

[nathalia@aws] $  less NA12878.vcf | grep -v "^#" | wc -l
218

[nathalia@aws] $  bcftools view -s NA12878 -e 'GT=="RR"' -o test.vcf all.vcf.gz

[nathalia@aws] $  less test.vcf | grep -v "^#" | wc -l
101
```

Note: `screen` and `tmux` 

# Joint calling

https://useast.ensembl.org/index.html

Uner favourite genomes, select Human.
Under other assemblies, select the 37 reference
Click BioMart

Select Human databases; Select Structural, then under Gene select Chromosome, gene start, gene end, gene name

```
Chromosome/scaffold name	Gene start (bp)	Gene end (bp)	Gene name
HG991_PATCH	66119285	66465398	SLC25A26
13	23551994	23552136	AL157931.1
13	23708313	23708703	HMGA1P6
13	23726725	23726825	RNY3P4
13	23743974	23744736	LINC00362
13	23791571	23791673	RNU6-58P
13	23817659	23821323	TATDN2P3
13	93708910	93710179	LINC00363
13	23841645	23843289	SDAD1P4
13	23947562	23948200	RPLP1P13

```

Download to local machine as tsv, then scp into AWS

```
[nathalia@cshl] $  scp -i /Users/seqtec/cshl/cshl_2018_student.pem mart_export-2.txt ubuntu@52.26.253.73:/home/ubuntu/workspace/cshl-practical/analysis/
mart_export-2.txt                             100% 1955KB   1.9MB/s   00:01    


[nathalia@aws] $  emacs mart_export-2.txt 

#delete two header lines, saved

[nathalia@aws] $  mv mart_export-2.txt genes.bed

[nathalia@aws] $  bedtools intersect -a genes.bed -b all.vcf.gz | cut -f 4 | less

# if want the column with gene names, so -a should be the bed file

[nathalia@aws] $  bedtools intersect -a genes.bed -b all.vcf.gz | cut -f 4 | uniq | less

[nathalia@aws] $  bedtools intersect -a genes.bed -b NA12878.vcf | cut -f 4| uniq | less

# these are the genes this particular sample has variants in

```

```
[nathalia@aws] $  bcftools view -s NA12878 all.vcf.gz | bcftools view -e 'GT=="RR"' -O z -o NA12878.vcf.gz -

[nathalia@aws] $  bcftools index NA12878.vcf.gz 

[nathalia@aws] $  tabix -p vcf NA12878.vcf.gz

[nathalia@aws] $  lhtr
total 30M
-rw-rw-r-- 1 ubuntu ubuntu  7.4M Nov 11 16:50 NA12878.bam
-rw-rw-r-- 1 ubuntu ubuntu  6.5M Nov 11 16:53 NA12891.bam
-rw-rw-r-- 1 ubuntu ubuntu  6.9M Nov 11 16:55 NA12892.bam
-rw-rw-r-- 1 ubuntu ubuntu 1007K Nov 11 16:56 NA12878.bam.bai
-rw-rw-r-- 1 ubuntu ubuntu  803K Nov 11 16:56 NA12891.bam.bai
-rw-rw-r-- 1 ubuntu ubuntu  1.1M Nov 11 16:56 NA12892.bam.bai
-rw-rw-r-- 1 ubuntu ubuntu   412 Nov 11 16:57 NA12878.stats
-rw-rw-r-- 1 ubuntu ubuntu   412 Nov 11 16:57 NA12891.stats
-rw-rw-r-- 1 ubuntu ubuntu   412 Nov 11 16:57 NA12892.stats
-rw-rw-r-- 1 ubuntu ubuntu  900K Nov 11 17:57 chr17.vcf
-rw-rw-r-- 1 ubuntu ubuntu   81K Nov 11 17:57 chr20.vcf
-rw-rw-r-- 1 ubuntu ubuntu  196K Nov 11 17:58 chr21.vcf
-rw-rw-r-- 1 ubuntu ubuntu  216K Nov 11 17:58 chr22.vcf
-rw-rw-r-- 1 ubuntu ubuntu  347K Nov 11 18:04 all.vcf.gz
-rw-rw-r-- 1 ubuntu ubuntu   352 Nov 11 18:04 all.vcf.gz.csi
-rw-rw-r-- 1 ubuntu ubuntu  116K Nov 11 18:40 NA12878.vcf
-rw-rw-r-- 1 ubuntu ubuntu   60K Nov 11 18:44 test.vcf
-rw-rw-r-- 1 ubuntu ubuntu   557 Nov 11 19:27 all.vcf.gz.tbi
-rw-r--r-- 1 ubuntu ubuntu  2.0M Nov 11 20:51 mart_export-2.txt~
-rw-r--r-- 1 ubuntu ubuntu  2.0M Nov 11 20:51 genes.bed
-rw-rw-r-- 1 ubuntu ubuntu   32K Nov 11 21:02 NA12878.vcf.gz
-rw-rw-r-- 1 ubuntu ubuntu   316 Nov 11 21:02 NA12878.vcf.gz.csi
-rw-rw-r-- 1 ubuntu ubuntu   503 Nov 11 21:04 NA12878.vcf.gz.tbi
```

Loaded the files into https://vcf.iobio.io/?species=Human&build=GRCh37&vcf=http%3A%2F%2Fec2-52-26-253-73.us-west-2.compute.amazonaws.com%2Fworkspace%2Fcshl-practical%2Fanalysis%2FNA12878.vcf.gz&tbi=

![join_calling](joint_calling_visualization.png)